# Convert Laz to DEM using CloudCompare

# Required software

+ CloudCompare
+ QGIS
+ GoogleColab

## Process flow

https://jeffblackadar.github.io/maryland_rch/maryland_rch-lidar.html

## File work areas

There are two file works areas
+ (local drive) local PC: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files
```
+ (Google Drive) Google Drive: 
```
/content/drive/MyDrive/crane_maryland/
```

### Download files from Google Drive to local drive

There is a constraint that only a 2gb zip file can be downloaded at one time. The .laz files are large so they have been batched into units of 50.
Download each batch.

### Local processing

The local PC has CloudCompare and QGIS installed, these are not available on Google Colab so it's necessary to moved files from Google Drive and process them on a PC and then move them back. Manual tasks but I could not find a better means to process these files using this collection of software.

On the local PC laz files will be converted to DEM using CloudCompare. CloudCompare is not installed on Google Colab so it's used on the local PC.

4 batches of 50 laz files (ex: 0-49, 50-99, 100-149, 150-199) will be downloaed into a folder that can contain 200 files (ex: laz_utm_0-199).

Steps:

#### 1. Create folders to hold downladed files. Example: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_0-199. 
```

This can be done by running the first cell of 1_1_Maryland_download_laz_tiles_use_CloudCompare_to_convert_to_DEM (below)


In [ ]:
# Cell 1. Create working directories
# Run this only once
# set up working directories to hold downloaded files
import os
input_laz_utm_fp_base = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"

chunk_amount = 200
total_files = 2154

for count in range(0, total_files , 200):

    new_dir_file_path = os.path.join(input_laz_utm_fp_base, ("laz_utm_"+str(count)+"-"+str(count+chunk_amount-1)))
    print(new_dir_file_path)
    if not (os.path.exists(new_dir_file_path)):  
        print("Does not exist.")
        os.mkdir(new_dir_file_path)
    else:
        print("Exists.")
        
        
input_slope_fp_base = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\"
for count in range(0, total_files , 200):

    new_dir_file_path = os.path.join(input_slope_fp_base, ("slope_"+str(count)+"-"+str(count+chunk_amount-1)))
    print(new_dir_file_path)
    if not (os.path.exists(new_dir_file_path)):  
        print("Does not exist.")
        os.mkdir(new_dir_file_path)
    else:
        print("Exists.")


#### 2. Manually download the files from Google Drive. Enter the folder and select the files. Verify 50 items selected.
(example):

```
/content/drive/MyDrive/crane_maryland/laz/laz_fema_2012_forested_utm/batch_50/laz_utm_0-49 
```

![local_processing_step_2.png](local_processing_step_2.png)

#### 3. Unzip into directory the appropriate directory to hold 200 files
Extract all

(example) from the zip file of 50 files:

```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-249-20211114T195617Z-001.zip\laz_utm_200-249
```

(example) to the folder to hold a group of 200 files: 
```
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399
```

Download and unzip 4 groups of 50 files into each folder to contain 200 files. Check that the folder has 200 files at the end of each group.

## Run CloudCompare

In [ ]:
# Cell 2 Run CloudCompare
# E:\Program Files\CloudCompare>cloudcompare -O 
#'E:/a_new_orgs/carleton/hist5706-maryland/lidar_files/reprojected/20120323_001415_r_utm.laz' 
#-SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1

import subprocess
import time

# example command
# C:\LAStools\bin\las2las -i "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\20120323_001460.laz" -target_utm 18north -o "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120323_001460_r_utm.laz"

batch_group = "1600-2199"
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"

counter = 0
import os
#process .laz files one at a time
for root, dirs, files in os.walk(input_laz_utm_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".laz"):
          laz_in_path = os.path.join(input_laz_utm_fp, name)
          #raster_out_path = os.path.join(output_raster_fp, (name[:-4]+".tif"))
          command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
          print(str(counter))        
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          #time.sleep(4)
          counter = counter + 1        
        

"""
Rasterizes the loaded clouds by projecting them inside a regular grid.
Optional settings are:

-VERT_DIR {0/1/2} to specify the projection dimension (0=X / 1=Y / 2=Z) - default is 2 (Z)
-PROJ {MIN/AVG/MAX} to specify how the altitude of each cell is computed (minimum, average or maximum altitude of the projected points). Default is AVG (average)
-SF_PROJ {MIN/AVG/MAX} to specify how the SF value(s) of each cell are computed (minimum, average or maximum altitude of the projected points SF value(s)). Default is AVG (average)
-EMPTY_FILL {MIN_H/MAX_H/CUSTOM_H/INTERP} to specify how empty cells should be filled (MIN_H = min height / MAX_H = max height / CUSTOM_H = custom value - see CUSTOM_HEIGHT option below / INTERP = interpolate). Default behavior is 'leave cells empty'
-CUSTOM_HEIGHT {value} to define the custom height filling value (if the 'CUSTOM_H' strategy is used (see above)
-OUTPUT_CLOUD to output the result as a cloud (this is the default option if no other output format is defined)
-OUTPUT_MESH to output the result as a mesh
-OUTPUT_RASTER_Z to output the result as a geotiff raster (only altitudes, no RGB)
-OUTPUT_RASTER_Z_AND_SF to output the result as a geotiff raster (altitudes + all SFs by default, no RGB)
-OUTPUT_RASTER_RGB to output the result as a geotiff raster (RGB)        
"""

# Re-run a skipped command individually

In [ ]:
!"E:\\Program Files\\CloudCompare\\cloudcompare" -SILENT -O E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_200-399\\20120218_000003_utm.laz -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z 

# Copy/Move tifs

CloudCompare creates the DEM tifs in the same directory as the las files. This program copies them to their own directory. Later, once this process is more repeatable, this will become a move, not a copy, to save disk space.

## !! Be careful and set the directories correctly.  Mixing batches together is annoying

In [ ]:
# Cell 3 Copy/Move tifs
import os
import shutil
# Divide files into groups of 200 so they can be downloaded and processed in managably sized chunks.
file_counter = 0

batch_group = "1600-2199"
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

if not (os.path.exists(dem_tif_fp_dest)):  
    print("Does not exist: ", dem_tif_fp_dest)
    os.mkdir(dem_tif_fp_dest)

for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            shutil.move(origin_file_path, dem_tif_fp_dest)
            print(file_counter, origin_file_path)
            file_counter = file_counter + 1        

# Quality Assurance 1: Confirm we have 200 lazs and 200 DEM tifs with matching file names.

In [6]:
# Cell 4 Quality Assurance 1
# Dictionary of lazs
batch_group = "1600-2199"
expected_number_of_files = 554
input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

lazs_dict = {}
for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".laz"):
            lazs_dict[file_name[:file_name.find("_utm")+4]]=file_name

# Dictionary of tifs
tifs_dict = {}
for file_name in os.listdir(dem_tif_fp_dest):
    origin_file_path = os.path.join(dem_tif_fp_dest,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            tifs_dict[file_name[:file_name.find("_utm")+4]]=file_name

counter = 0
print("There are:",len(lazs_dict),"laz files in", input_laz_utm_fp)
print("There are:",len(tifs_dict),"tif files in", dem_tif_fp_dest)
for dict_key in lazs_dict:
    if not dict_key in tifs_dict:
        print(lazs_dict[dict_key], " (laz) is not in tifs directory.")
        print("Let's try to process with a scalar field index of 6. For some Laz files, Classfication is at the index of 6, not 8.")
        laz_in_path = os.path.join(input_laz_utm_fp, lazs_dict[dict_key])
        # The index of the scalar field is 6 this time -SET_ACTIVE_SF 6
        command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
        print("file",str(counter), "processed.")
        print("!"+command)
        subprocess.call(command, shell=True)
        counter = counter + 1        


for dict_key in tifs_dict:
    if not dict_key in lazs_dict:
        print(tifs_dict[dict_key], "(tif) is not in lazs directory.")
      
if len(lazs_dict)==expected_number_of_files and len(tifs_dict)==expected_number_of_files:
    print("Both directories have",expected_number_of_files,"files. This looks good.")
else:
    print("There is a problem. Check the number of files in each folder.")

        

There are: 554 laz files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1600-2199\
There are: 554 tif files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1600-2199\dem_tifs\
Both directories have 554 files. This looks good.


# Open QGIS, Run:
```
1_2_qgis_load_dem_save_as_slope.py
```
This saves a slope file for each DEM tif in the diectory

```
1_3_qgis_save_slope_tif_as_jpg.py
```
This saves a jpg file for each slope tif in the diectory

# Quality Assurance 2: Confirm we have 200 lazs, 200 DEM tifs, 200 Slope tifs and 200 Slope jpgs with matching file names.

In [7]:
# Cell 5 Quality Assurance
# Dictionary of lazs

input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\"
dem_tif_fp_dest = os.path.join("E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_"+batch_group+"\\dem_tifs\\")

lazs_dict = {}
for file_name in os.listdir(input_laz_utm_fp):
    origin_file_path = os.path.join(input_laz_utm_fp,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".laz"):
            lazs_dict[file_name[:file_name.find("_utm")+4]]=file_name

# Dictionary of tifs
tifs_dict = {}
for file_name in os.listdir(dem_tif_fp_dest):
    origin_file_path = os.path.join(dem_tif_fp_dest,file_name)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            tifs_dict[file_name[:file_name.find("_utm")+4]]=file_name

counter = 0
print("There are:",len(lazs_dict),"laz files in", input_laz_utm_fp)
for dict_key in lazs_dict:
    if not dict_key in tifs_dict:
        print(lazs_dict[dict_key], "(laz) is not in tifs directory.")
        print("Go back to cell 4 and re-run the process. All of the lazs need to be converted to DEM tifs before running QGIS to create slope files.")
        print("file",str(counter), "processed.")
        counter = counter + 1        

print("There are:",len(tifs_dict),"tif files in", dem_tif_fp_dest)
for dict_key in tifs_dict:
    if not dict_key in lazs_dict:
        print(tifs_dict[dict_key], "(tif) is not in lazs directory.")        

# Dictionary of slope tifs
slope_tifs_dict = {}
slope_tifs_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\slope_"+batch_group+"\\"

for file_name in os.listdir(slope_tifs_fp):
    origin_file_path = os.path.join(slope_tifs_fp,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):
        if(origin_file_path[-4:]==".tif"):
            slope_tifs_dict[file_name[6:-4]]=file_name

print("There are:",len(slope_tifs_dict),"slope files in", slope_tifs_fp)
for dict_key in slope_tifs_dict:
    if not dict_key in tifs_dict:
        print(dict_key, "(slope tif) is not in DEM tifs directory.")
        
# Dictionary of slope jpgs
slope_jpgs_dict = {}
slope_jpgs_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\slope_"+batch_group+"\\jpgs\\"

for file_name in os.listdir(slope_jpgs_fp ):
    origin_file_path = os.path.join(slope_jpgs_fp ,file_name)
    #print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".jpg"):
            slope_jpgs_dict[file_name[6:-4]]=file_name

print("There are:",len(slope_jpgs_dict),"slope jpg files in", slope_jpgs_fp)
for dict_key in slope_jpgs_dict:
    if not dict_key in slope_tifs_dict:
        print(dict_key, "(slope jpg) is not in slope tifs directory.")

if len(lazs_dict)==expected_number_of_files and len(tifs_dict)==expected_number_of_files and len(slope_tifs_dict)==expected_number_of_files and len(slope_jpgs_dict)==expected_number_of_files:
    print("All directories have",expected_number_of_files," files. This looks good.")
else:
    print("There is a problem. Check the number of files in each folder.")

There are: 554 laz files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1600-2199\
There are: 554 tif files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1600-2199\dem_tifs\
There are: 554 slope files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\slope_1600-2199\
There are: 554 slope jpg files in E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\slope_1600-2199\jpgs\
All directories have 554  files. This looks good.


#### Copy JPGs and Tifs to Google Drive
The jpgs are needed for Mask R-CNN recognition. The Tifs are needed to convert image coordinates back into geographic coordinates.

Copy the jpg files to Google drive so they can be used by Mask R-CNN.  

In E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope
drag a folder (example: 1600-2199) to

to My Drive/crane_maryland/slope_images/